###**Os Bilionários Mais Ricos do Mundo** 🤑

#####*Etapa 01:* Importação do Conjunto de Dados

In [26]:
# Importação de bibliotecas
import pandas as pd
import sqlalchemy

In [27]:
# Importando os módulos e métodos específicos que serão importantes
from sqlalchemy import create_engine, text

In [28]:
# Lendo o conjunto de dados que será utilizado
url = '/content/top_200_richest_person_world.csv'
dados = pd.read_csv(url, encoding='ISO-8859-1')

#####*Etapa 02:* Exploração e Manipulação do Conjunto de Dados

In [29]:
# Obtendo as informações principais do conjunto de dados
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   S. No.    200 non-null    int64 
 1   Rank      200 non-null    int64 
 2   Name      200 non-null    object
 3   Age       200 non-null    int64 
 4   Country   200 non-null    object
 5   Networth  200 non-null    object
 6   Industry  200 non-null    object
dtypes: int64(3), object(4)
memory usage: 11.1+ KB


In [30]:
# Excluindo a coluna de índices (desnecessária)
dados.drop(columns=['S. No.'], inplace=True)

In [31]:
# Alterando o formato da coluna 'Networth' para float
dados['Networth'] = dados['Networth'].str.replace('$', '')
dados['Networth'] = dados['Networth'].str.replace(' B', '').astype(float)

#####*Etapa 03:* Injeção do Conjunto de Dados no Banco de Dados (*SQLite*)

In [32]:
# Criando um banco de dados local e uma conexão
engine = create_engine('sqlite:///memory:')

In [33]:
# Inserindo o dataframe (dados) dentro do banco de dados
dados.to_sql('bilionarios', engine, index=False)

200

In [34]:
# Verificando se o processo foi bem sucedido
pd.read_sql_table('bilionarios', con=engine)

,Rank,Name,Age,Country,Networth,Industry
0,1,Bernard Arnault & family,75,France,233.0,Fashion & Retail
1,2,Elon Musk,52,United States,195.0,Automotive
2,3,Jeff Bezos,60,United States,194.0,Technology
3,4,Mark Zuckerberg,39,United States,177.0,Technology
4,5,Larry Ellison,79,United States,141.0,Technology
...,...,...,...,...,...,...
195,195,Lei Jun,54,China,10.9,Technology
196,195,Georg Schaeffler,59,Germany,10.9,Automotive
197,195,Marcel Herrmann Telles & family,74,Brazil,10.9,Food & Beverage
198,199,David Velez & family,42,Colombia,10.8,Finance & Investments


#####*Etapa 04:* Análise do Conjunto de Dados

In [35]:
# Descobrindo o setor com maior quantidade de bilionários (em Python)
dados['Industry'].value_counts()

,count
Industry,
Technology,40
Finance & Investments,32
Fashion & Retail,30
Diversified,15
Food & Beverage,15
Metals & Mining,12
Automotive,10
Energy,9
Real Estate,8


In [36]:
# Descobrindo o setor com maior quantidade de bilionários (em SQL)
query = text('''SELECT Industry, COUNT(*) AS Quantity
                FROM bilionarios GROUP BY Industry
                ORDER BY Quantity DESC''')
pd.read_sql(query, con=engine)

,Industry,Quantity
0,Technology,40
1,Finance & Investments,32
2,Fashion & Retail,30
3,Food & Beverage,15
4,Diversified,15
5,Metals & Mining,12
6,Automotive,10
7,Energy,9
8,Real Estate,8
9,Manufacturing,8


In [50]:
# Analisando o setor com maior média de patrimônio líquido (em Python)
df_industria_bilionarios = dados.groupby('Industry')['Networth'].mean().sort_values(ascending=False).to_frame()
df_industria_bilionarios.head()

,Networth
Industry,
Telecom,102.000000
Technology,43.470000
Fashion & Retail,38.850000
Diversified,38.606667
Automotive,34.860000


In [38]:
# Analisando o setor com maior média de patrimônio líquido (em SQL)
query = text('''SELECT Industry, AVG(Networth) AS Average_networth
                FROM bilionarios GROUP BY Industry
                ORDER BY Average_networth DESC
            ''')
pd.read_sql(query, con=engine)

,Industry,Average_networth
0,Telecom,102.000000
1,Technology,43.470000
2,Fashion & Retail,38.850000
3,Diversified,38.606667
4,Automotive,34.860000
5,Media & Entertainment,32.766667
6,Logistics,30.575000
7,Finance & Investments,26.643750
8,Food & Beverage,24.413333
9,Metals & Mining,23.083333


In [43]:
# Verificando a distribuição geográfica dos bilionários (em Python)
df_pais_bilionarios = dados['Country'].value_counts().sort_values(ascending=False).to_frame()

In [44]:
# Alterando o nome da coluna de quantidade de bilionários
df_pais_bilionarios.rename(columns={'count': 'Quantity'}, inplace=True)
df_pais_bilionarios.head()

,Quantity
Country,
United States,79
China,16
India,13
Russia,11
Germany,11


In [48]:
# Verificando a distribuição geográfica dos bilionários (em SQL)
query = text('''SELECT Country, COUNT(*) AS Quantity
                FROM bilionarios
                GROUP BY Country
                ORDER BY Quantity DESC''')
pd.read_sql(query, con=engine)

,Country,Quantity
0,United States,79
1,China,16
2,India,13
3,Russia,11
4,Germany,11
5,Hong Kong,7
6,France,7
7,Australia,5
8,Indonesia,4
9,United Kingdom,3


#####*Etapa 05:* Exportação dos Conjuntos de Dados

In [53]:
# Exportando o dataframe de distribuição do patrimônio por indústria
df_industria_bilionarios.to_csv('distribuicao_industria.csv')

In [58]:
# Exportando o datafram de distribuição do patrimônio por país
df_pais_bilionarios.to_csv('distribuicao_pais.csv')